# The new kid on the block: DuckDB

Version 1.0 of DuckDB was released in 2024, it's pretty new and probably a must for data scientists.

In my opinion it's one of the most effective ways to process tabular data.

## What is it?

In short, is a tool to run SQL queries against files and dataframes. The SQL dialect is extremely sophisticated and it can directly read and write pandas dataframes, CSV fils, parquet, JSON and others, locally of from URLs or S3.

You will have a complete lesson on SQL later on, and we'll meet again and see the pros and cons of thes files format.

For today, this in just an overview